In [ ]:
!pip install kaggle

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#data collection through kaggle

In [4]:
kaggle_dictionary = json.load(open("kaggle.json"))

In [6]:
#setup kaggle api key as environment variables
os.environ["KAGGLE_USERNAME"] = kaggle_dictionary["username"]
os.environ["KAGGLE_KEY"] = kaggle_dictionary["key"]


In [7]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 47% 12.0M/25.7M [00:00<00:00, 125MB/s]
100% 25.7M/25.7M [00:00<00:00, 187MB/s]


In [8]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
#unzip the dataset

In [9]:
with ZipFile("imdb-dataset-of-50k-movie-reviews.zip","r") as zip_ref:
  zip_ref.extractall()

In [10]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [ ]:
#loading the dataset

In [11]:
data = pd.read_csv("/content/IMDB Dataset.csv")

In [12]:
data.shape

(50000, 2)

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [18]:
data["sentiment"].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [19]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)

In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
#splitting the data into train and test data

In [24]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=52)

In [27]:
print(train_data.shape)
print(test_data.shape)


(45000, 2)
(5000, 2)


In [ ]:
#data pre-processing

In [ ]:
#tokenization

In [28]:
tokenizer = Tokenizer(num_words=5000)

In [29]:
tokenizer.fit_on_texts(train_data["review"])
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [30]:
print(X_test)

[[1917   16  729 ...   16   11 1060]
 [   0    0    0 ...  234  171  204]
 [   0    0    0 ...  412    4  513]
 ...
 [ 330   12    9 ...   42   12   17]
 [   0    0    0 ...    5    1   63]
 [   3  769    4 ...    8    1  176]]


In [31]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [32]:
print(Y_train)

27343    0
17618    0
35323    0
48656    0
11793    0
        ..
41463    1
4247     1
43787    0
42652    1
23541    0
Name: sentiment, Length: 45000, dtype: int64


In [ ]:
#LSTM - Long Short Term Memory - useful for sequential dataset

In [33]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 128)          640000    
                                                                 
 lstm (LSTM)                 (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 771713 (2.94 MB)
Trainable params: 771713 (2.94 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
#compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [39]:
model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
563/563 [==============================] - 400s 701ms/step - loss: 0.3759 - accuracy: 0.8320 - val_loss: 0.3165 - val_accuracy: 0.8653
Epoch 2/5
563/563 [==============================] - 375s 665ms/step - loss: 0.2610 - accuracy: 0.8951 - val_loss: 0.3054 - val_accuracy: 0.8729
Epoch 3/5
563/563 [==============================] - 371s 659ms/step - loss: 0.2213 - accuracy: 0.9122 - val_loss: 0.2912 - val_accuracy: 0.8749
Epoch 4/5
563/563 [==============================] - 376s 668ms/step - loss: 0.1839 - accuracy: 0.9275 - val_loss: 0.3407 - val_accuracy: 0.8766
Epoch 5/5
563/563 [==============================] - 375s 667ms/step - loss: 0.1624 - accuracy: 0.9380 - val_loss: 0.3338 - val_accuracy: 0.8737


In [40]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

157/157 [==============================] - 9s 60ms/step - loss: 0.3312 - accuracy: 0.8756
Test Loss: 0.3311918377876282
Test Accuracy: 0.8755999803543091


In [41]:
def predict_sentiment(review):
  # tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
  return sentiment

In [42]:
new_review = "This movie was horrible. I hated it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 296ms/step
The sentiment of the review is: negative


In [43]:
new_review = "This movie was fantastic. I loved it."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 97ms/step
The sentiment of the review is: positive


In [44]:
new_review = "This movie gave me the shivers!!"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")


1/1 [==============================] - 0s 69ms/step
The sentiment of the review is: negative


In [45]:
new_review = "i fell in love once again!"
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is: {sentiment}")

1/1 [==============================] - 0s 67ms/step
The sentiment of the review is: positive
